In [167]:
# Imports
import numpy as np
import plotly.express as px
from PIL import Image

In [168]:
# Function to transform the output to a number between 0 and 1
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [169]:
# Load MNIST data
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [170]:
# Use only 1 and 0
train_mask = [True if y in [0, 1] else False for y in y_train]
test_mask = [True if y in [0, 1] else False for y in y_test]

x_train = x_train[train_mask]
x_test = x_test[test_mask]
y_train = y_train[train_mask]
y_test = y_test[test_mask]

In [172]:
# Input shape
input_shape = 9

In [173]:
# Preprocess the data
x_train = np.array([np.asarray(Image.fromarray(x).resize((input_shape,input_shape))) for x in x_train])
x_train = x_train / 250
x_test = np.array([np.asarray(Image.fromarray(x).resize((input_shape,input_shape))) for x in x_test])
x_test = x_test / 250

In [174]:
# Inspect one of the samples
px.imshow(x_train[0])

In [175]:
# Transform input to flat
def flatten_image(input):
  return input.reshape(input_shape*input_shape)

In [176]:
# Define the model function
def my_model(input, parameters):
    return sigmoid(sum(parameters*input))

In [177]:
# Define loss function
def loss_function(true_values, predictions):
    return sum((true_values - predictions) ** 2)

In [178]:
# Define function for a hidden layer
# Once for every node it runs the previous model
# It returns one number for each of it's node
def hidden_layer(input, parameters, nodes):
  parameters = parameters.reshape((nodes, input.shape[0]))

  output = []
  for i in parameters:
    output.append(my_model(input, i))

  return np.array(output)

In [179]:
# Put together the neural network by running several layers and return just one output
def my_neural_network(input, parameters):
  input_array = flatten_image(input)
  
  hidden_1 = hidden_layer(input_array, parameters[0], nodes=5)
  hidden_2 = hidden_layer(hidden_1, parameters[1], nodes=10)

  output = my_model(hidden_2, parameters[2])

  return output

In [180]:
# Specify the parameters we will use
# One array per layer
# Each array is as long as the number of nodes times the input shape
parameters = [np.ones(input_shape*input_shape*5), np.ones(5*10), np.ones(10)]

In [181]:
# Make sure it runs
my_neural_network(x_train[0], parameters=parameters)

0.9999514598637211

In [182]:
# Try with random parameters instead
parameters = [np.random.normal(size=input_shape*input_shape*5), np.random.normal(size=5*10), np.random.normal(size=10)]

In [183]:
# Make sure it runs
my_neural_network(x_train[0], parameters=parameters)

0.8710410947138629

In [184]:
# Define training function for the neural network
# This time we will just try random weights
# We will keep track of them with the help of a random seed
def train_nn(seed):
  np.random.seed(seed)
  parameters = [np.random.normal(size=input_shape*input_shape*5), np.random.normal(size=5*10), np.random.normal(size=10)]

  predictions = np.array([my_neural_network(x, parameters) for x in x_train])
  true_values = y_train

  loss = loss_function(true_values, predictions)

  return loss

In [185]:
# Let's try 100 different parameters set-ups
results = []
for seed in range(100):
  loss = train_nn(seed)
  results.append(loss)

In [186]:
# Inspect the various losses we got
px.scatter(x=range(100), y=results)

In [187]:
# Let's pick one of the seeds which gave us the lowest loss
np.random.seed(np.argmin(results))
parameters = [np.random.normal(size=input_shape*input_shape*5), np.random.normal(size=5*10), np.random.normal(size=10)]

In [188]:
# Get all predictions
predictions = np.array([my_neural_network(x, parameters) for x in x_test])
predictions = [1 if i > 0.5 else 0 for i in predictions]

In [189]:
# Accuracy
np.mean(predictions==y_test)

In [193]:
# Inspect some examples
for i in range(15):
  print("True value", y_test[i])
  print("Prediction", my_neural_network(x_test[i], parameters=parameters))
  px.imshow(x_test[i]).show()
  

True value 1
Prediction 0.573338812435561


True value 0
Prediction 0.38381994088618415


True value 1
Prediction 0.5921773937017978


True value 0
Prediction 0.27899919874359735


True value 0
Prediction 0.25035659178262515


...

True value 1
Prediction 0.5033683048424479


True value 1
Prediction 0.5552733418900608


True value 1
Prediction 0.5496443577623177


True value 1
Prediction 0.46149460631676537


True value 0
Prediction 0.4588524868829549
